In [ ]:
import tensorflow as tf
import numpy as np
import math
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
import keras
import livelossplot
import pandas as pd
import json
import umap.umap_ as umap
import sklearn.cluster as cluster

In [ ]:
genre = pd.read_csv('genre_labels.csv')
songs = pd.read_csv('song_order.txt', names = ['song_id'])
songs_with_genre = set(genre['song_id'])

In [ ]:
with open('full_normalized_song_features_128.json') as f: #song metadata and raw audio vectorised data
    song_data = json.load(f)

In [ ]:
inp_np = []
c = 0
for i in sorted(song_data):
    if i in songs_with_genre:
        inp_np.append(np.array(song_data[i]))
        c += 1

out_np = genre.sort_values(by='song_id').reset_index(drop=True).iloc[:,1]
genre_set = list(set(out_np))

genre_to_encode = {}

for ind, gen in enumerate(genre_set):
    genre_to_encode[gen] = ind

for i in range(out_np.shape[0]):
    out_np[i] = genre_to_encode[out_np[i]]

inp_np = np.array(inp_np).astype(np.float) #input vectors for the model
out_np = np.array(out_np).astype(np.float) #corresponding labels

Classification Model Architecture

In [ ]:
model= tf.keras.Sequential([
  tf.keras.layers.Dense(inp_np.shape[1], activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(1024, activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(15, activation='softmax')
])

In [ ]:
plot_losses1 = livelossplot.PlotLossesKeras()
model.compile(
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'],
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.005))



In [ ]:
model.fit(inp_np,out_np,
                callbacks=[plot_losses1],
                verbose=1,
                batch_size=32, epochs=20,validation_split = 0.2)

In [ ]:
model.summary()

Data Visualisation using UMAP and K-means

In [ ]:
standard_embedding = umap.UMAP(random_state=42).fit_transform(inp_np[:,:64])
plt.scatter(standard_embedding[:, 0], standard_embedding[:, 1], c=out_np.astype(int), s=0.1, cmap='Spectral')


In [ ]:
kmeans_labels = cluster.KMeans(n_clusters=14).fit_predict(inp_np)
plt.scatter(standard_embedding[:, 0], standard_embedding[:, 1], c=kmeans_labels, s=0.1, cmap='Spectral');